# **Run this after all the GP notebooks**

### This copy all the summary file and merge them by tagging _anion and _opensheel to the corresponding species properties
### Then it convert from Tom naming convenction to Leah naming convention

In [1]:
import os
import json
import pandas as pd
import argparse

summary_filename = 'Summary_Properties_all.xlsx'
summary_filename_merged = 'Summary_Properties_all_species_merged.xlsx'
postprocessed_results_folder = '5.postprocessed_results'

anion_summary_file = os.path.join('..', 'gp_anion', postprocessed_results_folder, summary_filename)
closed_shell_summary_file = os.path.join('..', 'gp_closed_shell', postprocessed_results_folder, summary_filename)
openshell_summary_file = os.path.join('..', 'gp_openshell', postprocessed_results_folder, summary_filename)

In [2]:
# check if these file exists, if not throw an error
if not os.path.exists(anion_summary_file):
    raise FileNotFoundError(f'{anion_summary_file} not found')
if not os.path.exists(closed_shell_summary_file):
    raise FileNotFoundError(f'{closed_shell_summary_file} not found')
if not os.path.exists(openshell_summary_file):
    raise FileNotFoundError(f'{openshell_summary_file} not found')

anion_summary_df = pd.read_excel(anion_summary_file)
display(anion_summary_df.head(1))
closed_shell_summary_df = pd.read_excel(closed_shell_summary_file)
display(closed_shell_summary_df.head(1))
openshell_summary_df = pd.read_excel(openshell_summary_file)
display(openshell_summary_df.head(1))

,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,...,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrdz1,-302.982492,0.088762,-302.887089,0.035899,0.035845,-302.922934,298.15,-0.02285,0.17703,...,3.280044,1.5108,1.2951,0.2156,3.0584,0.6203,-0.2574,3.3362,0.4938,-0.156


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,...,pyramidalization_Agranat-Radhakrishnan_C1_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrdz1,-303.585861,0.103728,-303.475215,0.036948,0.036617,-303.511832,298.15,-0.30541,-0.02096,...,0.798456,0.9572,0.9355,0.0217,3.2503,0.6982,0.0083,3.6184,0.313,-0.0686


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,...,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz
0,pyrdz1,-302.930027,0.090329,-302.833243,0.036086,0.036089,-302.869332,298.15,-0.26295,-0.02245,...,330.617365,0.937532,-0.25254,0.02923,1.4073,84.75594,95.861312,6.652501,1.700245,3.261611


In [3]:
# Initialize merged_df with Compound_Name column
anion_remaining = anion_summary_df.copy()
closed_shell_remaining = closed_shell_summary_df.copy()
openshell_remaining = openshell_summary_df.copy()
merged_df = closed_shell_summary_df[['Compound_Name']].copy()
merged_df.set_index('Compound_Name', inplace=True)

# Set Compound_Name as the index for all dataframes to enable row-wise merging
anion_remaining.set_index('Compound_Name', inplace=True)
closed_shell_remaining.set_index('Compound_Name', inplace=True)
openshell_remaining.set_index('Compound_Name', inplace=True)

# define column to skip
exclude_columns = ["T"]

# Process each column in closed_shell_remaining
for column in closed_shell_remaining.columns:
    # Add column from closed_shell_remaining to merged_df
    merged_df[column] = closed_shell_remaining[column]

    # Check and add columns from anion_remaining
    if column in anion_remaining.columns:
        if column not in exclude_columns:
            anion_col_df = anion_remaining[[column]].rename(columns={column: f"{column}_anion"})
            merged_df = merged_df.merge(anion_col_df, how='left', left_index=True, right_index=True)
        anion_remaining.drop(columns=[column], inplace=True)

    # Check and add columns from openshell_remaining
    if column in openshell_remaining.columns:
        if column not in exclude_columns:
            openshell_col_df = openshell_remaining[[column]].rename(columns={column: f"{column}_openshell"})
            merged_df = merged_df.merge(openshell_col_df, how='left', left_index=True, right_index=True)
        openshell_remaining.drop(columns=[column], inplace=True)

# Add remaining columns from anion_remaining
for column in anion_remaining.columns:
    remaining_anion_df = anion_remaining[[column]].rename(columns={column: f"{column}_anion"})
    merged_df = merged_df.merge(remaining_anion_df, how='left', left_index=True, right_index=True)

# Add remaining columns from openshell_remaining
for column in openshell_remaining.columns:
    remaining_openshell_df = openshell_remaining[[column]].rename(columns={column: f"{column}_openshell"})
    merged_df = merged_df.merge(remaining_openshell_df, how='left', left_index=True, right_index=True)

# Reset the index for the final dataframe
merged_df.reset_index(inplace=True)

# Display the resulting dataframe
display(merged_df.head())

,Compound_Name,E_spc (Hartree),E_spc (Hartree)_anion,E_spc (Hartree)_openshell,ZPE(Hartree),ZPE(Hartree)_anion,ZPE(Hartree)_openshell,H_spc(Hartree),H_spc(Hartree)_anion,H_spc(Hartree)_openshell,...,C1_Electro-Valency_Boltz,C1_Electro-Valency_Boltz_anion,C1_FormalCharge_Boltz,C1_FormalCharge_Boltz_anion,C2_Co-Valency_Boltz,C2_Co-Valency_Boltz_anion,C2_Electro-Valency_Boltz,C2_Electro-Valency_Boltz_anion,C2_FormalCharge_Boltz,C2_FormalCharge_Boltz_anion
0,pyrdz1,-303.585861,-302.982492,-302.930027,0.103728,0.088762,0.090329,-303.475215,-302.887089,-302.833243,...,0.698200,0.6203,0.00830,-0.2574,3.618400,3.3362,0.313000,0.4938,-0.068600,-0.1560
1,pyrdz2,-303.584349,-302.995475,-302.930289,0.103775,0.089531,0.090406,-303.473550,-302.899447,-302.833406,...,0.709900,0.5608,0.02410,-0.3223,3.814600,3.5204,0.122800,0.3562,-0.059800,-0.0593
2,pyrdz3,-342.892888,-342.289474,-342.243845,0.132713,0.116658,0.118646,-342.752044,-342.164508,-342.117063,...,0.475177,0.4730,-0.00134,-0.1756,3.601762,3.3759,0.314021,0.4329,-0.069421,-0.1479
3,pyrd1,-441.215801,-440.620456,-440.561372,0.163282,0.148368,0.150098,-441.043098,-440.463016,-440.402347,...,0.678400,0.5491,0.00060,-0.2896,3.537700,3.4047,0.417700,0.5367,-0.030800,-0.0333
4,pyrd2,-441.212895,-440.613489,-440.561090,0.163425,0.148142,0.150065,-441.040035,-440.456234,-440.402068,...,0.689000,0.6076,-0.00780,-0.2528,3.783700,3.5178,0.141900,0.3552,-0.046700,-0.0674


In [4]:
# write the merged dataframe to a new excel file
with pd.ExcelWriter(summary_filename_merged) as writer:
    merged_df.to_excel(writer, sheet_name="Summary_Properties_All", index=False)
    
    # automatically adjusts the width of the columns
    for column in merged_df.columns:
        column_width = max(merged_df[column].astype(str).map(len).max(), len(column))
        col_idx = merged_df.columns.get_loc(column)
        writer.sheets["Summary_Properties_All"].set_column(col_idx, col_idx, column_width)

In [5]:
# Read in the mapping from smiles_with_mapping.xlsx
df = pd.read_excel("smiles_with_mapping.xlsx", header=0)
df = df[["id", "mapping"]]

# Convert the mapping column to a dictionary with id being key, mapping being value
mapping_dict_from_tom_to_leah = dict(zip(df["id"], df["mapping"]))

In [6]:
mapping_dict_from_tom_to_leah

{'pyrd1': 'Het001',
 'pyrd2': 'Het002',
 'pyrd3': 'Het003',
 'pyrmd1': 'Het004',
 'pyrmd2': 'Het005',
 'pyrz1': 'Het006',
 'pyrd4': 'Het007',
 'pyrd5': 'Het008',
 'pyrd6': 'Het009',
 'pyrd7': 'Het010',
 'pyrd8': 'Het011',
 'pyrd9': 'Het012',
 'pyrmd3': 'Het013',
 'pyrmd4': 'Het014',
 'pyrz2': 'Het015',
 'pyrd10': 'Het016',
 'pyrd11': 'Het017',
 'pyrd12': 'Het018',
 'pyrd13': 'Het019',
 'pyrd14': 'Het020',
 'pyrd15': 'Het021',
 'pyrmd5': 'Het022',
 'pyrmd6': 'Het023',
 'pyrmd7': 'Het024',
 'pyrz3': 'Het025',
 'pyrdz1': 'Het026',
 'pyrdz2': 'Het027',
 'pyrd16': 'Het028',
 'pyrd17': 'Het029',
 'pyrd18': 'Het030',
 'pyrmd8': 'Het031',
 'pyrmd9': 'Het032',
 'pyrmd10': 'Het033',
 'pyrz4': 'Het034',
 'pyrdz3': 'Het035'}

In [7]:
#! define all the files you need to convert
files = [
    "Summary_Properties_all_species_merged.xlsx",
]

In [8]:
# read each file
for file in files:
    content = pd.read_excel(file, header=0)
    # display(content)
    # what we care is the Compound_Name column
    # go over row, use regular expression to match the column with the key from mapping_dict_from_tom_to_leah
    # if found, replace the value with the value from mapping_dict_from_tom_to_leah
    for index, row in content.iterrows():
        for key, value in mapping_dict_from_tom_to_leah.items():
            if key in row["Compound_Name"]:
                content.at[index, "Compound_Name"] = row["Compound_Name"].replace(
                    key, value
                )
    # then sort the row in ascend by Compound_Name, use this "\D+(\d+)" to find the number in the string
    content["Compound_numbering"] = content["Compound_Name"].str.extract("(\d+)", expand=False).astype(int)
    content = content.sort_values(by="Compound_numbering")
    content = content.drop(columns=["Compound_numbering"])
    # reset the index
    content = content.reset_index(drop=True)
    # display(content)
    # split the filename by "_" drop the last part, add "Leah" and join everything back
    new_filename = "_".join(file.split(".")[:-1]) + "_Leah.xlsx"
    display(f"saving to {new_filename}")
    # save the file
    with pd.ExcelWriter(new_filename) as writer:
        content.to_excel(writer, sheet_name="Summary_Properties_All", index=False)
        
        # automatically adjusts the width of the columns
        for column in content.columns:
            column_width = max(content[column].astype(str).map(len).max(), len(column))
            col_idx = content.columns.get_loc(column)
            writer.sheets["Summary_Properties_All"].set_column(col_idx, col_idx, column_width)

'saving to Summary_Properties_all_species_merged_Leah.xlsx'